In [1]:
import json

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
pd.options.display.max_columns = None

In [5]:
up_name = '纳豆奶奶'
up_id = '6574487'

# header转换函数 

In [6]:
def str_to_dict(x):
    return eval('{"' + x.replace(": ",'":"').replace('\n','","') + '"}')

# 探索-关注列表

In [7]:
headers = str_to_dict("""accept: */*
accept-encoding: gzip, deflate, br
accept-language: zh-CN,zh;q=0.9
referer: https://space.bilibili.com/6574487/fans/follow
sec-fetch-dest: script
sec-fetch-mode: no-cors
sec-fetch-site: same-site
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36""")

In [8]:
headers

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'zh-CN,zh;q=0.9',
 'referer': 'https://space.bilibili.com/6574487/fans/follow',
 'sec-fetch-dest': 'script',
 'sec-fetch-mode': 'no-cors',
 'sec-fetch-site': 'same-site',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [9]:
page = 1
api_url = "http://api.bilibili.com/x/relation/followings?vmid=6574487&pn=%s&ps=20&order=desc&jsonp=jsonp&callback=__jp5"%page

In [10]:
response = requests.get(api_url,headers=headers)

In [11]:
follows = json.loads(response.text[6:-1])

In [12]:
follows["data"]["list"]

[{'mid': 94114029,
  'attribute': 0,
  'mtime': 1603454978,
  'tag': [],
  'special': 0,
  'uname': '吃货请闭眼',
  'face': 'http://i2.hdslb.com/bfs/face/d895bb83306a81d36464627195739fff00150ddd.jpg',
  'sign': '中国那么大，我先吃一遍！微信公粽号：吃货请闭眼。淘宝搜店铺：吃货请闭眼小卖部。商务q3399679744（备注吃货合作）',
  'official_verify': {'type': 0, 'desc': 'bilibili 2019百大UP主、知名美食UP主'},
  'vip': {'vipType': 1,
   'vipDueDate': 1591891200000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 0,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},
 {'mid': 9099524,
  'attribute': 0,
  'mtime': 1602796867,
  'tag': [],
  'special': 0,
  'uname': '李旎CarlyLee',
  'face': 'http://i1.hdslb.com/bfs/app/6ad0e27720eeabd1c85873e82af5027a7994f911.jpg',
  'sign': 'destiny',
  'official_verify': {'type': 0, 'desc': 'bilibili副董事长兼COO'},
  'vip': {'vipType': 2,
   'vipDueDate': 1703260800000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 1,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},


# 抓取-关注列表

In [13]:
import time
import random

In [76]:
def follows_spider(up_id):
    data = []
    page = 1
    while page<=5:
        print(up_id,page)
        # 关注者api
        api_url = "http://api.bilibili.com/x/relation/followings?vmid=%s&pn=%s&ps=20&order=desc&jsonp=jsonp&callback=__jp5"%(up_id,page)
        response = requests.get(api_url,headers=headers)
        
        try:
            # 关注者
            follows = json.loads(response.text[6:-1])["data"]["list"]
        except Exception as e:
            print(e)
            print(json.loads(response.text[6:-1]))
        if not follows:
            break
            
        data.extend(follows)
        page += 1
        
        time.sleep(random.randint(6,11)*0.1)
    return data

In [15]:
follows_test = follows_spider(up_id)

6574487 2
6574487 3
6574487 4


In [16]:
follows_test

[{'mid': 94114029,
  'attribute': 0,
  'mtime': 1603454978,
  'tag': [],
  'special': 0,
  'uname': '吃货请闭眼',
  'face': 'http://i2.hdslb.com/bfs/face/d895bb83306a81d36464627195739fff00150ddd.jpg',
  'sign': '中国那么大，我先吃一遍！微信公粽号：吃货请闭眼。淘宝搜店铺：吃货请闭眼小卖部。商务q3399679744（备注吃货合作）',
  'official_verify': {'type': 0, 'desc': 'bilibili 2019百大UP主、知名美食UP主'},
  'vip': {'vipType': 1,
   'vipDueDate': 1591891200000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 0,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},
 {'mid': 9099524,
  'attribute': 0,
  'mtime': 1602796867,
  'tag': [],
  'special': 0,
  'uname': '李旎CarlyLee',
  'face': 'http://i1.hdslb.com/bfs/app/6ad0e27720eeabd1c85873e82af5027a7994f911.jpg',
  'sign': 'destiny',
  'official_verify': {'type': 0, 'desc': 'bilibili副董事长兼COO'},
  'vip': {'vipType': 2,
   'vipDueDate': 1703260800000,
   'dueRemark': '',
   'accessStatus': 0,
   'vipStatus': 1,
   'vipStatusWarn': '',
   'themeType': 0,
   'label': {'path': ''}}},


In [36]:
# 递归写法
def follow_tree_spider(seeds,deep=1):
    final_data = []
    
    if deep > 1:
        for i in range(deep):
            for seed in seeds:
                final_data += follows_spider(seed)
        return follow_tree_spider(pd.DataFrame(final_data)["mid"],deep-1)
    
    else:
        for seed in seeds:
            final_data += follows_spider(seed)
        return pd.DataFrame(final_data)

In [83]:
# 逐层搜索，避免递归
def follow_tree_spider(seeds,deep=1):
    # 详细数据
    final_data = pd.DataFrame()
    # 已获取
    cache = set()
    # 未获取
    queue = set(seeds)
    
    for i in range(deep):
        sub_data_raw = []
        sub_data = pd.DataFrame()
        for seed in queue:
            sub_data_raw.extend(follows_spider(seed))
#         print(sub_data_raw)
        sub_data = pd.DataFrame(sub_data_raw)
        # 单层搜索结束进行缓存更新
        cache.update(queue)
        # 当前列表去除缓存数据
#         print(sub_data.columns)
        queue = set(sub_data["mid"]) - cache
        # 数据汇总
        final_data = final_data.append(sub_data)
    return final_data

In [84]:
seeds = [up_id]

In [85]:
test = follow_tree_spider(seeds,2)

6574487 1
6574487 2
6574487 3
6574487 4
37090048 1
37090048 2
37090048 3
37090048 4
37090048 5
394813953 1
394813953 2
10119428 1
10119428 2
10119428 3
10119428 4
10119428 5
9099524 1
9099524 2
9099524 3
9099524 4
9099524 5
19642758 1
19642758 2
19642758 3
19642758 4
19642758 5
517327498 1
517327498 2
5465995 1
5465995 2
5465995 3
5465995 4
5465995 5
25782796 1
25782796 2
25782796 3
25782796 4
25782796 5
13354765 1
13354765 2
13354765 3
13354765 4
8366990 1
8366990 2
8366990 3
8366990 4
8366990 5
32708362 1
32708362 2
32708362 3
32708362 4
32708362 5
8047632 1
8047632 2
8047632 3
8047632 4
8047632 5
544368017 1
544368017 2
423442 1
423442 2
423442 3
423442 4
423442 5
546195 1
546195 2
120187282 1
120187282 2
120187282 3
120187282 4
120187282 5
1885078 1
1885078 2
1885078 3
1885078 4
1885078 5
168598 1
168598 2
168598 3
168598 4
168598 5
235555226 1
235555226 2
235555226 3
235555226 4
235555226 5
12076317 1
12076317 2
12076317 3
12076317 4
12076317 5
26366366 1
26366366 2
26366366 3
263

In [86]:
len(test),len(test.drop_duplicates("mid"))

(3612, 2022)

In [90]:
data_distinct = test.drop_duplicates("mid")

In [91]:
data_distinct.to_excel("follows_deep2.xlsx",index=False)

,mid,attribute,mtime,tag,special,uname,face,sign,official_verify,vip
0,94114029,0,1603454978,[],0,吃货请闭眼,http://i2.hdslb.com/bfs/face/d895bb83306a81d36...,中国那么大，我先吃一遍！微信公粽号：吃货请闭眼。淘宝搜店铺：吃货请闭眼小卖部。商务q3399...,"{'type': 0, 'desc': 'bilibili 2019百大UP主、知名美食UP主'}","{'vipType': 1, 'vipDueDate': 1591891200000, 'd..."
1,9099524,0,1602796867,[],0,李旎CarlyLee,http://i1.hdslb.com/bfs/app/6ad0e27720eeabd1c8...,destiny,"{'type': 0, 'desc': 'bilibili副董事长兼COO'}","{'vipType': 2, 'vipDueDate': 1703260800000, 'd..."
2,10119428,0,1593623166,[],0,小片片说大片,http://i1.hdslb.com/bfs/face/8c9c6777325ad761d...,VX公众号：小片片说大片。商务合作VX：xppsdp000,"{'type': 0, 'desc': 'bilibili 2019百大UP主、知名影视UP主'}","{'vipType': 2, 'vipDueDate': 1650384000000, 'd..."
3,345630501,0,1593451953,[],0,黄龄,http://i0.hdslb.com/bfs/face/292f54e57d34108f2...,龄铛交流平台—— QQ粉丝群：336405788\n商务合作：和序文化 林小姐 52902...,"{'type': 0, 'desc': '歌手 黄龄'}","{'vipType': 2, 'vipDueDate': 1615564800000, 'd..."
4,16206007,0,1592042853,[],0,vivi可爱多,http://i2.hdslb.com/bfs/face/10f7b3d3ff484c91b...,活的大胆一点，做想做的事。vb：@我是vivi可爱多。商务加微信：tiger36 注明【vi...,"{'type': 0, 'desc': 'bilibili 知名时尚UP主'}","{'vipType': 2, 'vipDueDate': 1631808000000, 'd..."
...,...,...,...,...,...,...,...,...,...,...
3537,20615621,0,1565103109,[],0,采紫葳的凌霄子,http://i1.hdslb.com/bfs/face/b3d6d9871475f15db...,谢谢你关注我，商务合作联系QQ：602156861，3群：805821028,"{'type': 0, 'desc': 'bilibili 知名鬼畜UP主、高能联盟成员'}","{'vipType': 2, 'vipDueDate': 1630425600000, 'd..."
3540,8230941,0,1562818693,[],0,愤怒六娃,http://i1.hdslb.com/bfs/face/d462a11324da12971...,一个专注映前的愤怒影评人，为观众趟平烂片雷区，推荐冷门院线佳片,"{'type': -1, 'desc': ''}","{'vipType': 1, 'vipDueDate': 1607270400000, 'd..."
3549,75,0,1555768006,[],0,丸子,http://i1.hdslb.com/bfs/face/8558700537e482f18...,还望关注+三连支持 商务合作请私信,"{'type': 0, 'desc': 'bilibili 直播签约主播'}","{'vipType': 2, 'vipDueDate': 2760451200000, 'd..."
3552,30605843,0,1551113000,[],0,原来是坨肉,http://i2.hdslb.com/bfs/face/f6f245e11210f83a9...,,"{'type': -1, 'desc': ''}","{'vipType': 1, 'vipDueDate': 1599667200000, 'd..."
